In [1]:
import numpy as np
import pandas as pd
import serial

In [2]:
# Format: Object-Distance-Height
class_names = ["Coin-10cm-10cm", "Coin-10cm-30cm", "Coin-30cm-10cm", "Coin-30cm-30cm"
               "Eraser-10cm-10cm", "Eraser-10cm-30cm", "Eraser-30cm-10cm", "Eraser-30cm-30cm"]

# Data Collection
Modify # Parameters

In [ ]:
# Using serial to read data from a serial port - using correct port
ser = serial.Serial(port="COM5", baudrate=230400, timeout=1) 
ser.flush()  # Flush the serial buffer

# Function to read ADC value from the serial port
def read_adc_value():
    try:
        if ser.in_waiting > 0:
            line = ser.readline().decode('utf-8').strip()
            return int(line)
    except ValueError:
        pass
    return None

# Function to save data to a CSV file
def save_to_csv(data, label, filename="data.csv"):
    df = pd.DataFrame(data, columns=["data" + str(i + 1) for i in range(len(data[0]))])
    df.insert(0, "label", label)  # Add the label column at the beginning
    df.to_csv(filename, index=False)

# Parameters
adc_threshold = 500
num_items_to_collect = 100  # Number of items to collect
class_label = class_names.index("Coin-10cm-10cm")

# Data collection
data = []

try:
    print("Data collection started...")
    while len(data) < num_items_to_collect:
        adc_value = read_adc_value()
        if adc_value is not None and adc_value > adc_threshold:
            # Collect data
            data.append([adc_value])  # Store ADC values as rows
    print("Data collection completed.")
except KeyboardInterrupt:
    print("Data collection interrupted.")
except Exception as e:
    print(f"An error occurred: {e}")

# Save data to CSV
if data:
    save_to_csv(data, class_label)
    print("Data saved to CSV.")
else:
    print("No data collected.")


SerialException: could not open port 'COM5': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)